In [1]:
import pandas as pd
import math as mt
import re

## auxillary functions

In [2]:
def get_claims(df: pd.DataFrame, id: int) -> pd.DataFrame:
    claims_text = df.loc[(df['Publn_Nr'] == id) & (df['Type'].str.match('Claim')), 'Text']  # get claim text
    return claims_text

In [3]:
def concantenate_claims(claims: pd.Series) -> str:
    return ' '.join(claims)

## apply NLP

In [4]:
names = ["FW", "LC", "LIB",  "SC", "SPV", "Wind"]  # technology file names
n = "Wind"

In [5]:
df = pd.read_excel(f'sorted_data/DB_{n}_USPTO_sorted.xlsx')
df['Publn_Nr'] = df['Publn_Nr'].astype(int)

In [6]:
# test, delete me
id = 10563637
claims_text = get_claims(df, id)
' '.join(claims_text)

'In operation, a wind power generating Twin M-VAWT have effective cooperation and interaction between paired planet and sun airfoils, as recited in claim 1, as well as having effective cooperation and interaction between 1.sup.st M-VAWT and 2.sup.nd M-VAWT, wherein: a) For each 1.sup.st M-VAWT and 2.sup.nd M-VAWT, multiple planet rotors and paired planet and sun airfoils rotate reciprocally through 2 half rotations separated by a line parallel to Y axis and through the sun stationary shaft; an outboard planet rotor is a planet rotor rotating through the half rotations further away from the Y axis, and its paired planet and sun airfoils are in half a rotation power cycles;  an inboard planet rotor is a planet rotor rotating through the half rotations closer to the Y axis, and its paired planet and sun airfoils are in half a rotation return cycles;  outboard planet rotors and their paired planet and sun airfoils rotate aft ward in +Y direction and away from generally straight flowing win

In [7]:
def checkRoman(token):
    re_pattern = '[mdcxvi]+[a-z]'
    if re.fullmatch(re_pattern, token):
        return True
    return False

In [8]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
import operator
import re

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
patent_claims = []  # list to store all patents claims for technology for year
list_replace = []
stwrds = stopwords.words('english')
stemmer = SnowballStemmer('english')


In [11]:
def get_tokens(text) -> list:
    # Standardize greek letters and eliminate symbols
    line = text.strip().lower()
    # Standardize greek letters and eliminate symbols
    for r in list_replace:
        line = line.replace(*r)
    # Replace .sub. and .sup. in each patent
    line = line.replace('.sub.', '')
    line = line.replace('.sup.', '')
    # Extract tokens using a regular expression
    tokens = re.findall('[a-z0-9][a-z0-9-]*[a-z0-9]+|[a-z0-9]', line)
    tokens = set(tokens)
    # Remove stopwords, and words of only one char and compossed only
    # of numbers
    tokens = [token for token in tokens if len(token) > 1 and
                token not in stwrds and
                not token.replace('-', '').isnumeric()]
    tokens = [stemmer.stem(token) for token in tokens]
    tokens = set(tokens)
    tokens = list(tokens)
    tokens.sort()
    tokens = [token for token in tokens if len(token) > 1 and
                token not in stwrds and
                not checkRoman(token)]
    return tokens

In [12]:
# count words in vocabulary
d_words = {}  
for tokens in patent_claims:
    for token in tokens:
        d_words[token] = d_words.get(token, 0) + 1

In [13]:
voc_sorted = sorted(d_words.items(), key=operator.itemgetter(1), reverse=True)
voc_sorted[:10]

[]

In [14]:
# get claims ordered by publication date
for pub_nr in df['Publn_Nr'].unique():
    claims = get_claims(df, pub_nr)
    if claims.empty:
        continue
    text = concantenate_claims(claims)  # concantenate claims
    tokens = get_tokens(text)           # get tokens